Assignment: Segmenting and Clustering Neighborhoods in Toronto



In [1]:
#import libraries
import pandas as pd
!pip install html5lib 
!pip install lxml
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

In [3]:
#drop rows where the Borough is Not Assigned
Wiki_table=df[df.Borough != 'Not assigned']

#Allow more than one neighborhood in each postal code area row:
Wiki_table = Wiki_table.groupby(['Postal Code','Borough'], as_index=False).agg(lambda x: ','.join(x))

#If a cell has a borough but a Not Assigned neighborhood, the neighborhood will be the same as the borough
mask=Wiki_table['Neighbourhood']=='Not assigned'
Wiki_table.loc[mask,'Neighbourhood']=Wiki_table.loc[mask,'Borough']




,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [20]:
Wiki_table.shape

(103, 3)

Assignment Part 2


In [23]:
#import geocoder
!pip install geocoder
import geocoder

#initialize variables to None
latitude=[]
longitude=[]

#loop through postal codes for lat long coordinates
for code in Wiki_table['Postal Code']:
    g=geocoder.arcgis('{},Toronto,Ontario'.format(code))
    
#loop until I get the coordinates
    while (g.latlng is None):
        g=geocoder.arcgis('{},Toronto,Ontario'.format(postal_code))
        print(code, g.latlng)
    latlng=g.latlng
    latitude.append(latlng[0])
    longitude.append(latlng[1])
      


In [25]:
Wiki_table['Latitude']=latitude
Wiki_table['Longitude']=longitude
Wiki_table.head()


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.78574,-79.15875
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17470
3,M1G,Scarborough,Woburn,43.76812,-79.21761
4,M1H,Scarborough,Cedarbrae,43.76944,-79.23892


Assignment Part 3


In [27]:
#explore boroughs that contain the word Toronto
scarborough_data = Wiki_table[Wiki_table['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head(7)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.78574,-79.15875
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17470
3,M1G,Scarborough,Woburn,43.76812,-79.21761
4,M1H,Scarborough,Cedarbrae,43.76944,-79.23892
5,M1J,Scarborough,Scarborough Village,43.74446,-79.23117
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.72582,-79.26461


In [29]:
#Create a map to visualize the Scarborough Neighborhoods

#enter Toronto lat and long
latitude_scar = 43.773077
longitude_scar = -79.257774

import folium

map_scar = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scar)  
    
map_scar

In [30]:
#Utilize the Goursquare API to explore and segment neighborhoods

CLIENT_ID = 'MEU2NOJRSIVPRRR234AZC1MWCZ4JEBOHWQLC2KMLR5DL5EM1' 
CLIENT_SECRET = 'H5KICJ2VMGVSBMQCR0MXTLTF4CG1UNSPSMTBXZHTRYP3HR0W'
VERSION = '20180605' 
LIMIT = 100 



Explore the first neighborhood in the dataframe


In [47]:
scarborough_data.loc[0,'Neighbourhood']

#Get the neighborhoods lat and long values
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighbourhood'] # neighborhood name
neighborhood_name

'Malvern, Rouge'

Get the top 100 venues in this neighborhood within a radius of 500m


In [48]:
import requests
import json

LIMIT=100
radius=500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fb732aef5f91049c06e7230'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.815890004500076,
    'lng': -79.19039569057789},
   'sw': {'lat': 43.80688999550007, 'lng': -79.20284430942199}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': 'Wendy’s',
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label':

Define the get_type_category function from the Foursquare lab


In [49]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean the json data and structure it into a pandas dataframe

In [50]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,name,categories,lat,lng
0,Wendy’s,Fast Food Restaurant,43.807448,-79.199056
1,Wood Bison Paddock,Zoo Exhibit,43.811732,-79.200708
2,Shyne Display Group,Furniture / Home Store,43.809040,-79.199596


Get venues for each neighborhood in Scarborough

In [51]:
#create function to repeat same process to all neighborhoods in Scarborough 
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [53]:
#new dataframe for scarborough venues
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighbourhood'],latitudes=scarborough_data['Latitude'],longitudes=scarborough_data['Longitude'])

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge


In [54]:
#check how many venues were returned for each neighborhood
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,15,15,15,15,15,15
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,1,1,1,1,1,1
"Clarks Corners, Tam O'Shanter, Sullivan",18,18,18,18,18,18
"Cliffside, Cliffcrest, Scarborough Village West",10,10,10,10,10,10
"Dorset Park, Wexford Heights, Scarborough Town Centre",2,2,2,2,2,2
"Golden Mile, Clairlea, Oakridge",10,10,10,10,10,10
"Guildwood, Morningside, West Hill",3,3,3,3,3,3
"Kennedy Park, Ionview, East Birchmount Park",3,3,3,3,3,3


Analyze Each Neighborhood


In [55]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,Auto Garage,Badminton Court,Bakery,Bank,Bar,Bubble Tea Shop,Bus Line,Bus Station,Bus Stop,...,Shopping Mall,Skating Rink,Soccer Field,Spa,Supermarket,Sushi Restaurant,Trail,Video Game Store,Vietnamese Restaurant,Zoo Exhibit
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group rows by neighborhood and take the mean frequency of occurrence of each category

In [57]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,Auto Garage,Badminton Court,Bakery,Bank,Bar,Bubble Tea Shop,Bus Line,Bus Station,Bus Stop,...,Shopping Mall,Skating Rink,Soccer Field,Spa,Supermarket,Sushi Restaurant,Trail,Video Game Store,Vietnamese Restaurant,Zoo Exhibit
0,Agincourt,0.0,0.066667,0.066667,0.000000,0.0,0.066667,0.0,0.0,0.000000,...,0.066667,0.066667,0.0,0.000000,0.066667,0.066667,0.0,0.000000,0.066667,0.000000
1,"Birch Cliff, Cliffside West",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.250000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,Cedarbrae,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.0,0.000000,0.055556,0.055556,0.0,0.000000,0.0,0.0,0.055556,...,0.000000,0.000000,0.0,0.055556,0.000000,0.000000,0.0,0.055556,0.000000,0.000000
4,"Cliffside, Cliffcrest, Scarborough Village West",0.0,0.000000,0.100000,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.0,0.000000,0.500000,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
6,"Golden Mile, Clairlea, Oakridge",0.0,0.000000,0.200000,0.000000,0.0,0.000000,0.2,0.1,0.000000,...,0.000000,0.000000,0.1,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
7,"Guildwood, Morningside, West Hill",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
8,"Kennedy Park, Ionview, East Birchmount Park",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
9,"Malvern, Rouge",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.333333


Print each neighborhood along with the top 5 most common venues

In [60]:
num_top_venues = 5

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                   venue  freq
0     Chinese Restaurant  0.13
1          Grocery Store  0.07
2                 Bakery  0.07
3  Vietnamese Restaurant  0.07
4   Hong Kong Restaurant  0.07


----Birch Cliff, Cliffside West----
                   venue  freq
0        College Stadium  0.25
1  General Entertainment  0.25
2           Skating Rink  0.25
3                   Café  0.25
4             Restaurant  0.00


----Cedarbrae----
                  venue  freq
0                 Trail   1.0
1           Auto Garage   0.0
2            Playground   0.0
3  Hong Kong Restaurant   0.0
4        Ice Cream Shop   0.0


----Clarks Corners, Tam O'Shanter, Sullivan----
                  venue  freq
0  Fast Food Restaurant  0.11
1           Pizza Place  0.11
2         Deli / Bodega  0.06
3  Cantonese Restaurant  0.06
4        Discount Store  0.06


----Cliffside, Cliffcrest, Scarborough Village West----
                venue  freq
0      Ice Cream Shop   0.2
1  Chinese Restaurant   0.1


Put this into a Pandas Dataframe

In [61]:
#function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [66]:
#create the dataframe and display the top 10 venues for each neighborhood
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Shanghai Restaurant,Hong Kong Restaurant,Badminton Court,Bakery,Sushi Restaurant,Supermarket,Grocery Store,Skating Rink,Shopping Mall
1,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Zoo Exhibit,Coffee Shop,Golf Course,Gift Shop,Furniture / Home Store,Fast Food Restaurant
2,Cedarbrae,Trail,Zoo Exhibit,Chinese Restaurant,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store,Fast Food Restaurant,Discount Store,Department Store
3,"Clarks Corners, Tam O'Shanter, Sullivan",Fast Food Restaurant,Pizza Place,Pharmacy,Sandwich Place,Grocery Store,Hobby Shop,Discount Store,Deli / Bodega,Coffee Shop,Cantonese Restaurant
4,"Cliffside, Cliffcrest, Scarborough Village West",Ice Cream Shop,Pizza Place,Chinese Restaurant,Bakery,Hardware Store,Discount Store,Sandwich Place,Coffee Shop,Pharmacy,General Entertainment


Cluster Neighborhoods


In [83]:
# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 3, 0, 0, 0, 0, 1, 0, 4], dtype=int32)

In [123]:
#create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood
# add clustering labels

#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_).reset_index(drop=True)
neighborhoods_venues_sorted

scarborough_merged = scarborough_data

scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
scarborough_merged
scarborough_merged=scarborough_merged.dropna()
scarborough_merged['Cluster Labels'].astype(int)



scarborough_merged = scarborough_merged.astype({"Cluster Labels": int})
scarborough_merged.dtypes

Postal Code                object
Borough                    object
Neighbourhood              object
Latitude                  float64
Longitude                 float64
Cluster Labels              int64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

Visualize the cluster


In [124]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
import matplotlib.cm as cm
import matplotlib.colors as colors

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighbourhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters